## Preprocessing

In [1386]:
# Import the required modules
import pandas as pd
import numpy as np

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold

# Deep Learning!
import tensorflow as tf

# Metrics
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve, RocCurveDisplay

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier


In [1387]:
# Import pandas and read the charity_data.csv from the provided cloud URL.
df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [1388]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = df.drop(columns=["EIN", "NAME"])

# Display the first few rows to confirm changes
print(df.shape)
df.head()

(34299, 10)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [1389]:
# Determine the number of unique values in each column.
unique_values = df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [1390]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
# Check the value counts of APPLICATION_TYPE
application_type_counts = df['APPLICATION_TYPE'].value_counts()
print(application_type_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [1391]:
# Determine the threshold for grouping into "Other"
threshold = 500  

# Replace application types that appear less than the threshold with "Other"
df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].apply(
    lambda x: "Other" if application_type_counts[x] < threshold else x)

# Verify the changes
print(df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [1392]:
# Determine the cutoff value
cutoff = 500  

temp = df.APPLICATION_TYPE.value_counts()
application_types_to_replace = list(temp.loc[temp < cutoff].index)

# Replace application types in the dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app, "Other")

# Check if the replacement was successful
print(df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [1393]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
# Check the value counts of CLASSIFICATION
classification_counts = df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [1394]:
# Determine a cutoff value
cutoff = 1000  

# Create a list of classifications to be replaced
classification_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace classifications in the dataframe
for cls in classification_to_replace:
   df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls, "Other")

# Verify the changes
print(df['CLASSIFICATION'].value_counts())


CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [1395]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# Check value counts of CLASSIFICATION and filter those appearing more than once
temp = df.CLASSIFICATION.value_counts()
temp.loc[temp > 1]
# Display classifications with counts greater than 1
classification_counts_filtered = classification_counts[classification_counts > 1]
print(classification_counts_filtered)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [1396]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Determine a cutoff value for classification
cutoff = 1000  

# Create a list of classifications to be replaced
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace classifications in the dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls, "Other")

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")

df_encoded = pd.get_dummies(df, dtype=int)
df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0


In [1397]:
# Check to make sure replacement was successful
df.IS_SUCCESSFUL.value_counts()

IS_SUCCESSFUL
1    18261
0    16038
Name: count, dtype: int64

In [1398]:
# Convert categorical variables to numeric using one-hot encoding
application_df = pd.get_dummies(df, dtype=int)

In [1399]:
# Split our preprocessed data into our features and target arrays
X = df_encoded.drop(['IS_SUCCESSFUL'], axis=1)
y = df_encoded['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 30, stratify=y)

print(X.shape)
print(X_train.shape)
print(X_test.shape)

(34299, 43)
(29154, 43)
(5145, 43)


In [1400]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [1401]:
def do_classification_nn(nn_model, X_train, X_test, y_train, y_test):
    # Predict probabilities
    train_proba = nn_model.predict(X_train)
    test_proba = nn_model.predict(X_test)
    
    # Convert probabilities to binary predictions
    train_preds = (train_proba >= 0.5).astype(int)
    test_preds = (test_proba >= 0.5).astype(int)
    
    # Ensure labels are NumPy arrays
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    # Compute metrics for training set
    train_cf = confusion_matrix(y_train, train_preds)
    train_cr = classification_report(y_train, train_preds, digits=4)
    train_auc = roc_auc_score(y_train, train_proba)
    
    train_results = f"""TRAIN METRICS
    Confusion Matrix:
    {train_cf}
    
    AUC: {train_auc:.4f}
    
    Classification Report:
    {train_cr}
    """
    
    # Compute metrics for test set
    test_cf = confusion_matrix(y_test, test_preds)
    test_cr = classification_report(y_test, test_preds, digits=4)
    test_auc = roc_auc_score(y_test, test_proba)
    
    test_results = f"""TEST METRICS
    Confusion Matrix:
    {test_cf}
    
    AUC: {test_auc:.4f}
    
    Classification Report:
    {test_cr}
    """
    
    print(train_results)
    print(test_results)
    
    return {
        "train": {
            "confusion_matrix": train_cf,
            "auc": train_auc,
            "classification_report": train_cr
        },
        "test": {
            "confusion_matrix": test_cf,
            "auc": test_auc,
            "classification_report": test_cr
        }
    }

In [1402]:
 # Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=len(X.columns)))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential_70"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_138 (Dense)               │ (None, 5)              │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_139 (Dense)               │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 226 (904.00 B)

 Trainable params: 226 (904.00 B)

 Non-trainable params: 0 (0.00 B)

In [1403]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [1404]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=15)

Epoch 1/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 2s 739us/step - accuracy: 0.4999 - loss: 7906.6343
Epoch 2/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 813us/step - accuracy: 0.5100 - loss: 7812.5454
Epoch 3/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 696us/step - accuracy: 0.5110 - loss: 9484.0898
Epoch 4/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 672us/step - accuracy: 0.5335 - loss: 5090.5747
Epoch 5/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 702us/step - accuracy: 0.5198 - loss: 5311.0142
Epoch 6/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 980us/step - accuracy: 0.5020 - loss: 3680.7871
Epoch 7/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 912us/step - accuracy: 0.5357 - loss: 4420.3418
Epoch 8/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 986us/step - accuracy: 0.5348 - loss: 8462.4189
Epoch 9/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 847us/step - accuracy: 0.5513 - loss: 16228.3760
Epoch 10/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - accuracy: 0.5228 - loss: 1715.7218
Epoch 11/15
912/912 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.5316 - loss: 2411.147

In [1405]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

161/161 - 0s - 970us/step - accuracy: 0.7026 - loss: 2.3658
Loss: 2.3657588958740234, Accuracy: 0.7026239037513733


In [1406]:
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity.h5')